In [1]:
import dash
import dash_core_components as dcc 
import dash_html_components as html 
import dash_table 
from dash_table.Format import Format, Scheme, Sign, Symbol
import dash_table.FormatTemplate as FormatTemplate
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go 
import plotly.figure_factory as ffimport 
import pandas as pd
import plotly.graph_objs as go
import numpy as np
import itertools
import json
import datetime as dt 

import sys
sys.path.append('..')

from app import app

ENABLE_WEBSOCKET_SUPPORT = True
refresh_rate = 5 if ENABLE_WEBSOCKET_SUPPORT else 7
if ENABLE_WEBSOCKET_SUPPORT:
    import diginex.ccxt.websocket_support as ccxt
else:
    import ccxt
    
# Define my world
deriv_exchanges =['deribit','bitmex']

exchanges =  deriv_exchanges 

api_keys = {'deribit':'4v58Wk2hhiG9B','bitmex':'DM55KFt84AfdjJiyNjDBk_km'}
api_secrets = {'deribit':'QLXLOZCOHAEQ6XV247KEXAKPX43GZLT4','bitmex':'jPDWiZcKuXJtVpajawENQiBzCKO2U885i3TWU9WIihBBUZgc'}

exch_dict={}
for x in exchanges:
    exec('exch_dict[x]=ccxt.{}()'.format(x))
for x,xccxt in exch_dict.items():
    xccxt.load_markets()
    xccxt.apiKey=api_keys[x]
    xccxt.secret=api_secrets[x]

deribit = exch_dict['deribit']
bitmex = exch_dict['bitmex']

Xpto = ['BTC','ETH']

In [2]:
def get_instruments():
    deribit_all_ins={}
    for base in Xpto:
        base_list=[]
        for ins in deribit.markets:
            market = deribit.markets[ins]
            if market['type']=='future' and market['base']==base:
                base_list.append(ins)
        deribit_all_ins[base]=base_list
    bitmex_all_ins={}
    for base in Xpto:
        base_list=[]
        for ins in bitmex.markets:
            market = bitmex.markets[ins]
            if market['type'] in ('future','swap') and market['base']==base and market['info']['hasLiquidity']:
                expiry = market['info']['expiry']
                if expiry is None:
                    base_list.append(ins)
                else:
                    dt_expiry = dt.datetime.strptime(expiry,"%Y-%m-%dT%H:%M:%S.%fZ")
                    if dt_expiry > dt.datetime.now():
                        base_list.append(ins)
        bitmex_all_ins[base]=base_list
    return deribit_all_ins,bitmex_all_ins

In [3]:
deribit_all_ins , bitmex_all_ins = get_instruments()
deribit_ins = deribit_all_ins['BTC']
bitmex_ins= bitmex_all_ins['BTC']
instruments = deribit_ins + bitmex_ins

In [4]:
def get_exchanges_for_pair(pair):
    '''input: a pair
    output: a dictionary of ccxt exchange objects of the exchanges listing the pair
    '''
    return {x:exch_dict[x] for x in exch_dict if pair in list(exch_dict[x].load_markets().keys())}

In [5]:
def get_pairs_for_exchange(ex):
    '''input: an exchange
    output: a list of pairs '''
    d={}
    exec('d[ex]=ccxt.{}()'.format(ex))
    d[ex].load_markets()
    return d[ex].symbols

In [6]:
def get_order_books(pair,ex):
    '''pair is the pair string ,'BTC/USD'...
        returns a dictionary of order books for the pair
        special case for binance which API fails if # of parmaeters > 2
    '''
    order_books = {key: value.fetch_order_book(pair,limit=2000 ,
                        params={'full':1,'level':3,'limit_bids':0,'limit_asks':0,'type':'both'})
                        for key,value in ex.items() }
    if 'deribit' in order_books:
        bids_df = pd.DataFrame(order_books['deribit']['bids'])
        asks_df = pd.DataFrame(order_books['deribit']['asks'])
        bids_df[1]=bids_df[1]*10
        asks_df[1]= asks_df[1]*10
        order_books['deribit']['bids']=bids_df.values.tolist()
        order_books['deribit']['asks']=asks_df.values.tolist()
    return order_books

In [ ]:
def aggregate_order_books(dict_of_order_books):
    '''dict_of_order_books is a dict of ccxt like order_books
        retuns a ccxt like dictionary order book sorted by prices 
    '''
    agg_dict_order_book = {}
    bids = []
    for x in dict_of_order_books:
        for bid in dict_of_order_books[x]['bids']:
            bids.append(bid+[x])
    asks = []
    for x in dict_of_order_books:
        for ask in dict_of_order_books[x]['asks']:
            asks.append(ask+[x])
    agg_dict_order_book['bids'] = (pd.DataFrame(bids)).sort_values(by=0,ascending=False).values.tolist()
    agg_dict_order_book['asks'] = (pd.DataFrame(asks)).sort_values(by=0,ascending=True).values.tolist()
    return agg_dict_order_book

In [ ]:
def normalize_order_book(order_book,cutoff=.1,step=.001):
    '''order_book is a dictionary with keys bids asks timestamp datetime ...
    where bids is a list of list [[bid,bid_size]] and 
    asks is a list of list [[ask,ask_size]]
    this is returned by ccxt.'exchange'.fetch_order_book()
    returns a dataframe with columns [ask, ask_size, ask_size_$, cum_ask_size_$, bid_, bid_size, bid_size_$, cum_bid_size_$]
    and an index of shape np.linspace(1 - cutoff,1 + cutoff ,step =.001 ~ 10 bps)  
    '''
    try:
        rounding = int(np.ceil(-np.log(step)/np.log(10)))
        agg = True
    except:
        agg = False
    bid_side = pd.DataFrame(order_book['bids'],columns=['bid','bid_size_$','exc'])
    bid_side['cum_bid_size_$'] = bid_side['bid_size_$'].cumsum()
    ask_side = pd.DataFrame(order_book['asks'],columns=['ask','ask_size_$','exc'])
    ask_side['cum_ask_size_$'] = ask_side['ask_size_$'].cumsum()
    ref = (bid_side['bid'][0]+ask_side['ask'][0])/2
    bid_side['bid%'] = round(bid_side['bid']/ref,rounding) if agg else bid_side['bid']/ref
    ask_side['ask%'] = round(ask_side['ask']/ref,rounding) if agg else ask_side['ask']/ref
    bid_side = bid_side[bid_side['bid%']>=1-cutoff]
    ask_side = ask_side[ask_side['ask%']<=1+cutoff]
    bid_side['bid_size'] = bid_side['bid_size_$']/bid_side['bid']
    bid_side['cum_bid_size'] = bid_side['bid_size'].cumsum()
    ask_side['ask_size'] = ask_side['ask_size_$']/ask_side['ask']
    ask_side['cum_ask_size'] = ask_side['ask_size'].cumsum()
    normalized_bids = pd.DataFrame(bid_side.groupby('bid%',sort=False).mean()['bid'])
    normalized_bids.columns = ['bid']
    normalized_bids['bid_size'] = bid_side.groupby('bid%',sort=False).sum()['bid_size']
    normalized_bids['cum_bid_size'] = normalized_bids['bid_size'].cumsum()
    normalized_bids['bid_size_$'] = bid_side.groupby('bid%',sort=False).sum()['bid_size_$']
    normalized_bids['cum_bid_size_$'] = normalized_bids['bid_size_$'].cumsum()
    normalized_bids['average_bid_fill'] = normalized_bids['cum_bid_size_$']/normalized_bids['cum_bid_size']
    normalized_bids['bids_exc']=bid_side.groupby('bid%',sort=False).apply(lambda x: x['exc'].loc[x['bid_size'].idxmax()])
    normalized_asks = pd.DataFrame(ask_side.groupby('ask%',sort=False).mean()['ask'])
    normalized_asks.columns = ['ask']
    normalized_asks['ask_size'] = ask_side.groupby('ask%',sort=False).sum()['ask_size']
    normalized_asks['cum_ask_size'] = normalized_asks['ask_size'].cumsum()
    normalized_asks['ask_size_$'] = ask_side.groupby('ask%',sort=False).sum()['ask_size_$']
    normalized_asks['cum_ask_size_$'] = normalized_asks['ask_size_$'].cumsum()
    normalized_asks['average_ask_fill']=normalized_asks['cum_ask_size_$']/normalized_asks['cum_ask_size']
    normalized_asks['asks_exc']=ask_side.groupby('ask%',sort=False).apply(lambda x: x['exc'].loc[x['ask_size'].idxmax()])
    book=pd.concat([normalized_asks,normalized_bids],sort=False)
    return book

In [ ]:
def build_book(order_books,pair,exchanges,cutoff=.1,step=0.001):
    ''' gets order books aggreagtes them then normalizes
        returns a dataframe
    '''
    return normalize_order_book(aggregate_order_books({key:order_books[key] for key in exchanges}),cutoff,step)

In [ ]:
pair = 'BTC-PERPETUAL'

In [ ]:
ex = get_exchanges_for_pair(pair)

In [ ]:
ex

In [ ]:
obs = get_order_books(pair,ex)

In [ ]:
obs['deribit']

In [ ]:
exch_dict

In [ ]:
exch_dict['bitmex'].markets

In [ ]:
def get_instruments():
    deribit_all_ins={}
    for base in Xpto:
        base_list=[]
        for ins in deribit.markets:
            market = deribit.markets[ins]
            if market['type']=='future' and market['base']==base:
                base_list.append(ins)
        deribit_all_ins[base]=base_list
    bitmex_all_ins={}
    for base in Xpto:
        base_list=[]
        for ins in bitmex.markets:
            if not ins.startswith('.') and '_' not in ins:
                market = bitmex.markets[ins]
                if market['type'] in ('future','swap') and market['base']==base:
                    expiry = market['info']['expiry']
                    if expiry is None:
                        base_list.append(ins)
                    else:
                        dt_expiry = dt.datetime.strptime(expiry,"%Y-%m-%dT%H:%M:%S.%fZ")
                        if dt_expiry > dt.datetime.now():
                            base_list.append(ins)
            bitmex_all_ins[base]=base_list
    return deribit_all_ins,bitmex_all_ins

In [ ]:
get_instruments()

In [ ]:
for ins in bitmex.markets:
    if not ins.startswith('.'):
        print (ins)

In [ ]:
bitmex.markets['XBT7D_U105']

In [ ]:
expiry=deribit.markets['BTC-28JUN19']['info']

In [ ]:
deribit.fetch_balance()

In [27]:
pair = 'BTC/USD'

In [28]:
ex = get_exchanges_for_pair(pair)

In [40]:
order_books_1 = {key: value.fetch_order_book(pair)
                        for key,value in ex.items() }

In [47]:
len(order_books_1['bitmex']['bids'])

3273

In [42]:
order_books_2 = {key: value.fetch_order_book(pair,limit=2000 ,
                        params={'full':1,'level':3,'limit_bids':0,'limit_asks':0,'type':'both'})
                        for key,value in ex.items() }

In [48]:
len(order_books_2['bitmex']['bids'])

2000

In [54]:
bitmex.fetch_open_orders()

[{'info': {'orderID': 'c7e65482-86ae-dfc9-426d-fb858f60d395',
   'clOrdID': '',
   'clOrdLinkID': '',
   'account': 787492,
   'symbol': 'XBTUSD',
   'side': 'Buy',
   'simpleOrderQty': None,
   'orderQty': 1,
   'price': 4112,
   'displayQty': None,
   'stopPx': None,
   'pegOffsetValue': None,
   'pegPriceType': '',
   'currency': 'USD',
   'settlCurrency': 'XBt',
   'ordType': 'Limit',
   'timeInForce': 'GoodTillCancel',
   'execInst': '',
   'contingencyType': '',
   'exDestination': 'XBME',
   'ordStatus': 'New',
   'triggered': '',
   'workingIndicator': True,
   'ordRejReason': '',
   'simpleLeavesQty': None,
   'leavesQty': 1,
   'simpleCumQty': None,
   'cumQty': 0,
   'avgPx': None,
   'multiLegReportingType': 'SingleSecurity',
   'text': 'Submission from www.bitmex.com',
   'transactTime': '2019-04-01T11:21:24.991Z',
   'timestamp': '2019-04-01T11:21:24.991Z'},
  'id': 'c7e65482-86ae-dfc9-426d-fb858f60d395',
  'timestamp': 1554117684991,
  'datetime': '2019-04-01T11:21:24.99

In [50]:
bitmex.fetch_closed_orders()

[]